In [62]:
from algorithm_ranking import MeasurementsVisualizer, MeasurementsManager, RankVariantsDFGTr, RankVariantsSort2
from am4pa.linnea import LinneaConfig, MeasurementsLinnea
from variants_compare import VariantsCompare
from pm4py.objects.conversion.log import converter as log_converter
import random
import os
import pandas as pd

In [2]:
linnea_config = LinneaConfig("GLS",12)
linnea_config.set_local_directory("linnea/GLS/")
linnea_config.setup_backend_details(server="login18-1.hpc.itc.rwth-aachen.de",
                                   uname = "as641651",
                                   init_script="~/.analyzer",
                                   backend_dir="~/PhD/va4algs-exps/GLS/")

In [3]:
linnea_config.connect_backend()

In [4]:
#linnea_config.bm.copy_to_backend(linnea_config.local_dir, linnea_config.backend_dir)

In [16]:
op_sizes = [["1000", "200"], ["1000", "300"], ["1000", "100"], ["1000", "150"] ]

In [17]:
def generate_variants(lc, ops):
    ms = []
    for i, op in enumerate(ops):
        ms.append(MeasurementsLinnea(lc, op))
        if not lc.bm.check_if_dir_exists(ms[i].runner.operands_dir):
            ms[i].generate_variants()
    return ms

In [18]:
ms_ = generate_variants(linnea_config, op_sizes)

source ~/.analyzer; cd ~/PhD/va4algs-exps/GLS; python generate-variants-linnea.py 1000 150 --threads=12
['New solution:..............6.8e+08\n', 'New solution:.............6.59e+08\n', 'New solution:.............5.08e+08\n', 'Nodes:.........................384\n', 'Nodes:.........................713\n', 'Nodes:........................1001\n', 'Nodes:........................1282\n', 'Nodes:........................1629\n', 'Nodes:........................1884\n', 'Nodes:........................2141\n', 'Nodes:........................2379\n', 'Nodes:........................2707\n', 'Nodes:........................2920\n', 'Nodes:........................3154\n', 'Nodes:........................3408\n', 'Nodes:........................3796\n', 'Nodes:........................4060\n', 'Nodes:........................4516\n', 'Nodes:........................4839\n', 'Nodes:........................5126\n', 'Nodes:........................5457\n', 'Nodes:........................5712\n', 'Nodes:........

In [21]:
def measure_variants(ms, reps = 5):
    for m in ms:
        h0 = m.gather_competing_variants(bmeasure_once=False, rel_duration=1.1)
        print(len(h0))
        m.measure(reps,1)

In [22]:
measure_variants(ms_, reps=5)

100
source ~/.analyzer; cd ~/PhD/va4algs-exps/GLS/experiments/1000_200; python generate-measurements-script.py --algs algorithm87 algorithm83 algorithm57 algorithm40 algorithm65 algorithm8 algorithm64 algorithm0 algorithm53 algorithm46 algorithm49 algorithm96 algorithm56 algorithm54 algorithm39 algorithm14 algorithm10 algorithm93 algorithm72 algorithm91 algorithm2 algorithm36 algorithm97 algorithm59 algorithm6 algorithm95 algorithm70 algorithm55 algorithm31 algorithm62 algorithm69 algorithm4 algorithm9 algorithm33 algorithm74 algorithm99 algorithm41 algorithm26 algorithm7 algorithm51 algorithm1 algorithm17 algorithm11 algorithm42 algorithm15 algorithm52 algorithm24 algorithm67 algorithm20 algorithm60 algorithm94 algorithm5 algorithm84 algorithm58 algorithm50 algorithm88 algorithm37 algorithm73 algorithm82 algorithm79 algorithm29 algorithm19 algorithm32 algorithm61 algorithm80 algorithm48 algorithm45 algorithm68 algorithm92 algorithm34 algorithm44 algorithm22 algorithm12 algorithm89 alg

In [113]:
def clean_concept(name):
    splits = name.split('=')
    if len(splits) > 1:
        return splits[-1].strip()
    return splits[0].strip()

def rank_variants_dfg(alg_measurements, alg_list):
    rv = RankVariantsDFGTr(alg_measurements, alg_list)
    return rv.rank_variants()

def rank_variants_dfg_reliable(alg_measurements, alg_list):
    rv = RankVariantsDFGTr(alg_measurements, alg_list)
    return rv.rank_variants_reliable()[0].iloc[:,:2]
    

def comparison_data(ms, cutoff=0, clear=False):
    vc = []
    data_nodes = []
    data_edges = []
    data_ext = []
    dbest_a = []
    dworst_a = []
    for i, m in enumerate(ms):
        if clear:
            m.data_collector.delete_local_data()
        m.case_durations_manager.clear_case_durations()
        m.collect_measurements(1)
        #dfs.append(rank_variants_dfg_reliable(m.get_alg_measurements(), m.h0))
        ranks = rank_variants_dfg_reliable(m.get_alg_measurements(), m.h0)
        best = ranks[ranks.iloc[:,1]<=cutoff]['case:concept:name'].tolist()
        worst = ranks[ranks.iloc[:,1]>cutoff]['case:concept:name'].tolist()
        
        dc = m.data_collector
        et = dc.get_meta_table()
        et['concept:name'] = et['concept:name'].apply(lambda row: clean_concept(row))
        
        for alg in best:
            dbest_a.append((et[et['case:concept:name']==alg]['concept:name'].apply(lambda x: x.split('_')[0]).tolist()))
        
        for alg in worst:
            dworst_a.append((et[et['case:concept:name']==alg]['concept:name'].apply(lambda x: x.split('_')[0]).tolist()))
            
        xes_log = log_converter.apply(et)
        
        activity_key = 'concept:name'
        vc.append(VariantsCompare(xes_log,best,worst,activity_key=activity_key))
        dn, de = vc[i].get_diff_data()
        ext = et.drop_duplicates(subset=['concept:name'])[['concept:name', 'concept:flops']]
        data_nodes.append(dn)
        data_edges.append(de)
        data_ext.append(ext)
        
    return pd.concat(data_nodes).reset_index(drop=True), \
           pd.concat(data_edges).reset_index(drop=True), \
           dbest_a, dworst_a
        #print(worst)

In [114]:
dn_, de_, db_, dw_ = comparison_data(ms_)

/Users/aravind/.local/lib/python3.9/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")
/Users/aravind/.local/lib/python3.9/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")
/Users/aravind/.local/lib/python3.9/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")
/Users/aravind/.local/lib/python3.9/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprec

In [115]:
dn_

,node,class
0,LAPACK.potrf_2.67e+06,0
1,LAPACK.potrf_3.33e+08,0
2,LAPACK.syev_2.67e+07,-1
3,gemm_8e+07,0
4,gemv_200,-1
5,gemv_4e+05,0
6,gemv_8e+04,-1
7,qr_2.13e+07,-1
8,syrk_4e+07,0
9,transpose_1,-1


In [117]:
de_

,nodeA,nodeB,class
0,LAPACK.potrf_2.67e+06,gemv_4e+05,-1
1,LAPACK.potrf_2.67e+06,trsm_4e+07,-1
2,LAPACK.potrf_2.67e+06,trsv_1e+06,0
3,LAPACK.potrf_2.67e+06,trsv_4e+04,0
4,LAPACK.potrf_3.33e+08,trsm_2e+08,0
...,...,...,...
203,trsv_1e+06,trsv_1e+06,-1
204,trsv_2.25e+04,trsv_2.25e+04,0
205,trsv_2.25e+04,@@endnode,0
206,@@startnode,LAPACK.potrf_3.33e+08,0


In [120]:
db_

[['LAPACK.potrf',
  'trsm',
  'trsv',
  'syrk',
  'gemv',
  'LAPACK.potrf',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsm',
  'syrk',
  'LAPACK.potrf',
  'trsv',
  'gemv',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsv',
  'trsm',
  'gemm',
  'gemv',
  'LAPACK.potrf',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsm',
  'syrk',
  'trsv',
  'gemv',
  'LAPACK.potrf',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsv',
  'trsm',
  'gemv',
  'gemm',
  'LAPACK.potrf',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsv',
  'trsm',
  'gemv',
  'syrk',
  'LAPACK.potrf',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsv',
  'trsm',
  'syrk',
  'gemv',
  'LAPACK.potrf',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsm',
  'trsv',
  'gemm',
  'gemv',
  'LAPACK.potrf',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsm',
  'trsv',
  'syrk',
  'gemv',
  'LAPACK.potrf',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsm',
  'syrk',
  'LAPACK.potrf',
  'trsv',
  'gemv',
  'trsv',
  'trsv'],
 ['LAPACK.potrf',
  'trsm',
  

In [121]:
def get_flops(str_):
    if not '@@' in str_:
        return float(str_.split('_')[1])
    return 0

def prepare_data(dn, de):
    dn['flops'] = dn.apply(lambda x: get_flops(x['node']), axis=1)
    dn['kernel'] = dn.apply(lambda x: x['node'].split('_')[0], axis=1)
    de['flopsA'] = de.apply(lambda x: get_flops(x['nodeA']), axis=1)
    de['kernelA'] = de.apply(lambda x: x['nodeA'].split('_')[0], axis=1)
    de['flopsB'] = de.apply(lambda x: get_flops(x['nodeB']), axis=1)
    de['kernelB'] = de.apply(lambda x: x['nodeB'].split('_')[0], axis=1)

In [122]:
prepare_data(dn_, de_)

In [123]:
de_

,nodeA,nodeB,class,flopsA,kernelA,flopsB,kernelB
0,LAPACK.potrf_2.67e+06,gemv_4e+05,-1,2670000.0,LAPACK.potrf,400000.0,gemv
1,LAPACK.potrf_2.67e+06,trsm_4e+07,-1,2670000.0,LAPACK.potrf,40000000.0,trsm
2,LAPACK.potrf_2.67e+06,trsv_1e+06,0,2670000.0,LAPACK.potrf,1000000.0,trsv
3,LAPACK.potrf_2.67e+06,trsv_4e+04,0,2670000.0,LAPACK.potrf,40000.0,trsv
4,LAPACK.potrf_3.33e+08,trsm_2e+08,0,333000000.0,LAPACK.potrf,200000000.0,trsm
...,...,...,...,...,...,...,...
203,trsv_1e+06,trsv_1e+06,-1,1000000.0,trsv,1000000.0,trsv
204,trsv_2.25e+04,trsv_2.25e+04,0,22500.0,trsv,22500.0,trsv
205,trsv_2.25e+04,@@endnode,0,22500.0,trsv,0.0,@@endnode
206,@@startnode,LAPACK.potrf_3.33e+08,0,0.0,@@startnode,333000000.0,LAPACK.potrf


In [124]:
dn_

,node,class,flops,kernel
0,LAPACK.potrf_2.67e+06,0,2670000.0,LAPACK.potrf
1,LAPACK.potrf_3.33e+08,0,333000000.0,LAPACK.potrf
2,LAPACK.syev_2.67e+07,-1,26700000.0,LAPACK.syev
3,gemm_8e+07,0,80000000.0,gemm
4,gemv_200,-1,200.0,gemv
5,gemv_4e+05,0,400000.0,gemv
6,gemv_8e+04,-1,80000.0,gemv
7,qr_2.13e+07,-1,21300000.0,qr
8,syrk_4e+07,0,40000000.0,syrk
9,transpose_1,-1,1.0,transpose
